In [5]:
import pandas as pd
df = pd.read_json("moonbase_reqs_2.json")
df.rename(columns={"definition_no_rerank": "definition"}, inplace=True)#
df['length'] = df['definition'].apply(lambda x: len(x))
df['triples'] = df['definition'].apply(lambda x: len(x.split("\n")))

In [2]:
import pandas as pd
df = pd.read_json("moonbase_reqs_2.json")
df

,req_id,requirement,definition
0,RQ-8-1,The settlement shall regenerate CO2 into O2,|Habitation| |is_contained_by| |Logical System...
1,RQ-8-2,The settlement shall generate oxygen using water,|Water Loop| |is_contained_by| |Logical System...
2,RQ-8-3,The settlement shall remove sufficient particl...,"|Structure, Shielding, & Storage| |is_containe..."
3,RQ-8-4,The settlement shall have at least 70% efficie...,|Waste Management| |is_contained_by| |Logical ...
4,RQ-8-5,The settlement shall maintain an atmosphere of...,|Air Loop| |is_contained_by| |Logical System|\...
5,RQ-2-1,The settlement shall provide food preparation ...,|Food Loop| |is_contained_by| |Logical System|...
6,RQ-2-2,The settlement shall provide private sleeping ...,|Habitation| |is_contained_by| |Logical System...
7,RQ-2-3,The settlement shall provide exercise facilities,|Settlement Peripherals & Payloads| |directly_...
8,RQ-2-4,The settlement shall provide medical supplies ...,|Emergency System| |is_contained_by| |Logical ...
9,RQ-2-5,The settlement shall provide sufficient medica...,|Emergency System| |is_contained_by| |Logical ...


In [6]:
# Load requirements
import os 
import json
script_dir = "./"
inputs = {}
inputs["input_path"] = "moonbase_reqs_2.json"
inputs["output_path"] = "results_gpt4_cot_moon.json"

with open(os.path.join(script_dir, inputs["input_path"]), "r") as f:
    requirements = json.load(f)

# Try to load output JSON and process it
try:    
    output_path = os.path.join(script_dir, inputs["output_path"])
    with open(output_path, "r") as f:
        output_data = json.load(f)

    # Extract req_ids from output_data
    # Assuming each item in the output_data is a dictionary with an 'id' key
    req_ids = set(item['req_id'] for item in output_data if 'req_id' in item)

    # Remove these ids from requirements
    # This assumes that requirements is a list of dicts with an 'id' key
    requirements = [req for req in requirements if req.get('req_id') not in req_ids]

except FileNotFoundError:
    print(f"Output file {output_path} not found.")
except json.JSONDecodeError:
    print("Error in reading or parsing the JSON file.")

In [10]:
with open(os.path.join(script_dir, inputs["input_path"]), "r") as f:
    requirements = json.load(f)

len(requirements)

49

In [9]:
len(requirements)

39

In [8]:
#req_ids

In [11]:
#output_data

In [69]:
#df

In [2]:
import tiktoken
def get_prompt_length(text, tokenizer):
    tokens = tokenizer.encode(text)
    num_tokens = len(tokens)
    return num_tokens

## Function

In [8]:
import openai
import time 
from test_reasoning_moon import create_prompt_string, extract_final_answer
def generate_openai(inputs, script_dir): 
    
    prompt_path = os.path.join(script_dir, inputs["prompt_path"])
    with open(prompt_path, 'r') as f:
        prompt_load = f.read()

    req = inputs['req']

    definition = inputs['definition']
    prompt_print = create_prompt_string(prompt_reason=prompt_load, user_prompt="", definition = definition, req= req, system_prompt ="", cot = inputs['cot']) #f"{prompt_reason}\n\n System:{definition} \nRequirement: {req} \nASSISTANT: Let's analyze the given System and requirement step-by-step:" 
    
    model =inputs['model']#"gpt-3.5-turbo-1106" #inputs['model']   

    _response = openai.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": prompt_print}, {"role": "assistant", "content": "Let's think step-by-step:"}],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    #stop=["Final Answer: Yes", "Final Answer: No"],#None,
    logprobs=True,
    top_logprobs = 3,
    n=1
    )
    time.sleep(0.5)

    response = _response.model_dump()#dict()

    results = {"requirement":req ,'reasoning':[], "split_flag": ""}
    score = np.exp(response['choices'][0]['logprobs']['content'][-1]['logprob'])
    output = response['choices'][0]['message']['content']

    final_answer, predict = extract_final_answer(output, cot=inputs['cot'], internal_cot=inputs["internal_cot"])

    results['reasoning'].append({"prompt": prompt_print, "output": output, "final_answer": final_answer, "predict": predict, "score": score})
       
    return results

In [10]:

import os 
import numpy as np 
import openai

from tqdm.notebook import tqdm

def set_environment_variables_from_file(file_path):
    with open(file_path) as file:
        for line in file:
            if line.strip() and not line.startswith('#'):
                key, value = line.strip().split('=', 1)
                os.environ[key] = value


# Set environment variables from the config file
set_environment_variables_from_file('config.env')
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
openai.api_key = OPENAI_API_KEY

requirements = df.to_dict(orient='records')
inputs = {}

dryrun = False# True # False #True# True #False#True
total_length = 0 
answers = []
ignore = 0

# To get the tokeniser corresponding to a specific model in the OpenAI API:
tokenizer = tiktoken.encoding_for_model("gpt-4")

responses = []
results = []
for req in tqdm(requirements[0:1]): 

    temp = {}
    ## define "True" test case 
    inputs['req'] = req['requirement']
    inputs['definition'] = req['definition']
    inputs['cot']= False
    inputs['internal_cot'] = True
    inputs['prompt_path'] = "./Machine_Learning/prompt_openchat.txt"
    # inputs['prompt']
    script_dir = "./"

    definition = inputs['definition']
    inputs['model'] = "gpt-3.5-turbo-1106"
    #req = inputs['req']

    prompt_path = os.path.join(script_dir, inputs["prompt_path"])
    with open(prompt_path, 'r') as f:
        prompt_load = f.read()

    prompt = create_prompt_string(prompt_reason=prompt_load, user_prompt="", definition = definition, req= inputs['req'], system_prompt ="", cot = inputs['cot'])
    total_length += get_prompt_length(prompt, tokenizer)

    if not dryrun:
        temp = generate_openai(inputs, script_dir)

        #score = np.exp(responses['choices'][0]['logprobs']['content'][-1]['logprob'])
        #output = responses['choices'][0]['message']['content']

        #temp = {"requirement":inputs['req'] ,'reasoning':[], "split_flag": ""}
        
        #temp['reasoning'].append({"prompt": prompt, "output": output, "final_answer": final_answer, "predict": predict, "score": score})
        #temp['req_id'] = req['req_id']
        results.append(temp)

print(f"Total length of the encoding: {total_length}")
print(f"Pricing for the encoding of prompt: {round(total_length * 0.01/ 1000,3)}$")

  0%|          | 0/1 [00:00<?, ?it/s]

Total length of the encoding: 907
Pricing for the encoding of prompt: 0.009$


In [11]:
results[0]

{'requirement': 'The settlement shall regenerate CO2 into O2',
 'reasoning': [{'prompt': 'System: |Habitation| |is_contained_by| |Logical System|\n|Habitation| |directly_performs| |Distribute Power|\n|Distribute Power| |Transmit Power| |Distribute Power|\n|Distribute Power| |is_performed_by| |Power|\n|Habitation| |directly_performs| |Provide Sleeping Quarters|\n|Provide Sleeping Quarters| |Move to Quarters| |Execute Work|\n|Execute Work| |is_performed_by| |Crew|\n|Provide Sleeping Quarters| |Setup| |Duplicate 75|\n|Duplicate 75| |is_performed_by| |Crew|\n|Provide Sleeping Quarters| |Go to bed| |Sleep|\n|Sleep| |is_performed_by| |Crew|\n|Habitation| |directly_performs| |Provide Exercise Facilities|\n|Provide Exercise Facilities| |Setup| |Duplicate 75|\n|Duplicate 75| |is_performed_by| |Crew|\n|Provide Exercise Facilities| |Move to Gym| |Exercise|\n|Exercise| |is_performed_by| |Crew|\n|Habitation| |directly_performs| |Provide Medical Services|\n|Provide Medical Services| |Move to Medical

In [ ]:
df

In [68]:
df_reasoning

,prompt,output,final_answer,predict,score,req_id
0,System: |Habitation| |is_contained_by| |Logica...,"First, we have the ""Air Loop"" within the ""Logi...",True,yes,0.99996,RQ-8-1


In [70]:
from pandas import json_normalize


data= results
df = pd.DataFrame(data)
# Normalize the nested 'reasoning' column
df_reasoning = json_normalize(results, 'reasoning', ['req_id'])#['requirement', 'split_flag', 'gt'])
# Merge the normalized reasoning DataFrame with the original DataFrame
#df_expanded = df.drop('reasoning', axis=1).merge(df_reasoning, on=['output'])#['requirement', 'split_flag', 'gt'])
df_expanded = df.drop('reasoning', axis=1).merge(df_reasoning, left_index=True, right_index=True)
# lowercase the output column
df_expanded['output'] = df_expanded['output'].apply(lambda x: x.lower())
df = df_expanded
df = df.drop('req_id_y', axis= 1)
df = df.rename(columns={'req_id_x':'req_id'})

In [71]:
df

,requirement,split_flag,req_id,prompt,output,final_answer,predict,score
0,The settlement shall regenerate CO2 into O2,,RQ-8-1,System: |Habitation| |is_contained_by| |Logica...,"first, we have the ""air loop"" within the ""logi...",True,yes,0.99996


In [46]:
responses

[]

In [18]:
total_length

907

## other stuff

In [ ]:
# output = response['choices'][0]['message']['content']
            # results['reasoning'].append({"prompt": prompt_print, "output": output, "final_answer": False, "predict": "too long", "score": "undefined"})
            # print(response)
        
        # while True:
        # try:

        # except Exception as e:
        #     print(e)
        #     if ("limit" in str(e)):
        #         time.sleep(15)
        #     else:
        #         ignore += 1
        #         raise e

## test

In [ ]:
import openai
import os 
import time 

from datetime import datetime

def set_environment_variables_from_file(file_path):
    with open(file_path) as file:
        for line in file:
            if line.strip() and not line.startswith('#'):
                key, value = line.strip().split('=', 1)
                os.environ[key] = value

# Set environment variables from the config file
set_environment_variables_from_file('config.env')
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
openai.api_key = OPENAI_API_KEY

# To get the tokeniser corresponding to a specific model in the OpenAI API:
tokenizer = tiktoken.encoding_for_model("gpt-4")

#model = "gpt-3.5-turbo-1106"
model = "gpt-4-1106-preview"

gpt_system_prompt = "You are a helpful assistant that always tries to answer requests to the best of its abilities."

dryrun = False# True #False#True
total_length = 0 
answers = []
ignore = 0
for row in df.index[:3]:

    print(row)
    question = df.loc[row, "question"]
    #context = df.loc[row, "context"]
    context = df.loc[row, "text_chunk"]
    ANSWER_FORMULATE = f"""\
    Answer the question using the information from the given context. 
    Question: {question}
    Context: {context}
    Answer:
    """  # noqa: E501
    #print(ANSWER_FORMULATE)
    total_length += get_prompt_length(ANSWER_FORMULATE, tokenizer)

    if not dryrun:
        
        while True:
            try:
                _response = openai.chat.completions.create(
                model=model,
                messages=[{"role": "system", "content": gpt_system_prompt}, {"role": "user", "content": ANSWER_FORMULATE}],#"Let's think step-by-step:"}],
                temperature=0.5,
                max_tokens=1000,
                top_p=0.95,
                frequency_penalty=0,
                presence_penalty=0,
                #stop=["Final Answer: Yes", "Final Answer: No"],
                #logprobs=10,
                n=1
                )
                answers.append(_response.choices[0].message.content)
                break
            except Exception as e:
                print(e)
                if ("limit" in str(e)):
                    time.sleep(15)
                else:
                    ignore += 1
                    raise e
                    print('ignored', ignore)
print(f"Total length of the encoding: {total_length}")
print(f"Pricing for the encoding of prompt: {round(total_length * 0.01/ 1000,3)}$")
length_generated =0
for answer in answers:
    length_generated += get_prompt_length(answer, tokenizer)

print(f"Total length of generated: {length_generated}")
print(f"Pricing for the encoding of generated: {round(length_generated * 0.03/ 1000,3)}$")
print(answers)